In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
# from pandas.tseries.offsets import BDay
# from pandas.tseries.offsets import MonthEnd
# import datetime

In [4]:
dfhold = pd.read_csv("../../../Data/vw_HoldActivity.csv",encoding='latin-1',low_memory=False)
dfhold.head()

,TicketNumber,ActivityId,RegardingObjectId,HoldDuration,Statuscode,Statecode,TimeZoneRuleVersionNumber,HoldTypeName,Reason,AssignedToGroup,StartTime,EndTime,IsThreeStrikeRule,HoldSubReason,Modified_On,Deleted
0,5-0000000464223,28E0D3AE-0907-E711-80E9-3863BB2ED230,2058B401-7A06-E711-80F3-FC15B428DE24,4620,Completed,Completed,NaN,3rd Party,Trade - Trade Status Hold,Internal,2017-03-12 09:53:03,2017-03-15 14:16:17,0,NaN,2017-03-15 14:16:17,NaN
1,5-0000000418488,79A45B63-3507-E711-80E9-3863BB2ED230,54A3CC57-50FE-E611-80F1-FC15B428DE24,0,Open,Open,NaN,Customer,Price Error,Submitter (Contact),2017-03-12 15:05:55,NaN,0,NaN,2017-03-12 15:05:55,NaN
2,5-0000000402148,241CB04D-3907-E711-80E9-3863BB2ED230,60D07540-BBFA-E611-80F6-FC15B4289B54,120,Completed,Completed,0.0,Customer,PO Number Error,Submitter (Contact),2017-03-12 15:33:56,2017-03-12 17:18:02,0,NaN,2017-03-28 06:15:33,NaN
3,5-0000000463272,996E4A98-5407-E711-80E9-3863BB2ED230,905C98C5-CB05-E711-80F3-FC15B4284D68,3300,Completed,Completed,0.0,Internal,Internal Hold - Team Lead,Submitter (Contact),2017-03-12 18:49:25,2017-03-15 01:46:18,0,NaN,2017-03-15 01:46:18,NaN
4,5-0000000408448,2A1CD3FE-5907-E711-80E9-3863BB2ED230,51E16092-0BFD-E611-80F1-FC15B428DE24,180,Completed,Completed,NaN,Internal,A - Missing information from customer - Confir...,Submitter (Contact),2017-03-12 19:28:00,2017-03-12 22:42:43,0,NaN,2017-03-12 22:42:43,NaN
